# Denpendency

In [ ]:
from google.colab import drive
import os
drive.mount('/content/drive/')
os.chdir('/content/drive/My Drive/Colabs')

Mounted at /content/drive/


In [ ]:
import re
import csv
import pandas as pd
from sklearn.model_selection import train_test_split

In [ ]:
import numpy as np
import csv
import torch
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import VotingClassifier
from argparse import ArgumentParser
from sklearn import metrics

In [ ]:
# Install the vncorenlp python wrapper
!pip install vncorenlp

# Download VnCoreNLP-1.1.1.jar & its word segmentation component (i.e. RDRSegmenter) 

# !mkdir -p vncorenlp/models/wordsegmenter
# !wget https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/VnCoreNLP-1.1.1.jar
# !wget https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/models/wordsegmenter/vi-vocab
# !wget https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/models/wordsegmenter/wordsegmenter.rdr
# !mv VnCoreNLP-1.1.1.jar vncorenlp/ 
# !mv vi-vocab vncorenlp/models/wordsegmenter/
# !mv wordsegmenter.rdr vncorenlp/models/wordsegmenter/

     |████████████████████████████████| 2.6 MB 9.4 MB/s 
  Created wheel for vncorenlp: filename=vncorenlp-1.0.3-py3-none-any.whl size=2645951 sha256=143feac9ec3b7d137634fe746da9e532290279a501d766ffa09b28dd4418a036
  Stored in directory: /root/.cache/pip/wheels/0c/d8/f2/d28d97379b4f6479bf51247c8dfd57fa00932fa7a74b6aab29
Successfully built vncorenlp


In [ ]:
from vncorenlp import VnCoreNLP
rdrsegmenter = VnCoreNLP("/content/drive/MyDrive/Colabs/vncorenlp/VnCoreNLP-1.1.1.jar", annotators="wseg", max_heap_size='-Xmx500m') 

text = "Đại học Quốc Gia Hà Nội"

word_segmented_text = rdrsegmenter.tokenize(text) 
print(word_segmented_text)

[['Đại_học', 'Quốc_Gia', 'Hà_Nội']]


# Load and clean data new data



In [ ]:
DATA_ROOT_DIR="/content/drive/MyDrive/Colabs"

TRAIN_FILE = "/content/drive/MyDrive/Colabs/data/train.crash"
TEST_FILE = "/content/drive/MyDrive/Colabs/data/test.crash"

TRAIN_CSV = "/content/drive/MyDrive/Colabs/data/train.csv"
TEST_CSV = "/content/drive/MyDrive/Colabs/data/test.csv"

DEV_TRAIN_CSV = "/content/drive/MyDrive/Colabs/data/train_dev.csv"
VAL_CSV = "/content/drive/MyDrive/Colabs/data/test_dev.csv"

In [ ]:
convert_word = {
    "ship": "vận chuyển",
    "shop": "cửa hàng",
    "m": "mình",
    "mik": "mình",
    "ko": "không",
    "k": " không ",
    "kh": "không",
    "khong": "không",
    "kg": "không",
    "khg": "không",
    "tl": "trả lời",
    "r": "rồi",
    "fb": "mạng xã hội",
    "face": "mạng xã hội",
    "thanks": "cảm ơn",
    "thank": "cảm ơn",
    "tks": "cảm ơn",
    "tk": "cảm ơn",
    "ok": "tốt",
    "dc": "được",
    "vs": "với",
    "đt": "điện thoại",
    "thjk": "thích",
    "thik":"thích",
    "qá": "quá",
    "trể": "trễ",
    "bgjo": "bao giờ",
    "h":"giờ",
    "qa":"quá",
    "dep":"đẹp",
    "xau":"xấu",
    "ib":"nhắn tin",
    "cute":"dễ thương",
    "sz":"size",
    "good":"tốt",
    "god":"tốt",
    "bt":"bình thường",
}


def tokmap(tok):
    if tok.lower() in convert_word:
        return convert_word[tok.lower()]
    else:
        return tok


def preprocess(review):
    review = review.lower()
    tokens = review.split()
    tokens = map(tokmap, tokens)
    return " ".join(tokens)


def load_data(filepath, is_train=True):
    regex = 'train_'
    if not is_train:
        regex = 'test_'

    item = []
    listItem = []

    with open(filepath, "r", encoding="utf-8", errors="ignore") as f:
        for line in f:
            line = line.strip()
            if regex in line:
                listItem.append(item)
                item = [line]
            elif line != "\n" and line != "":
                item.append(line)

        listItem.append(item)
    listItem = listItem[1:]
    lst = []
    for e in listItem:
        idx = e[0]
        if is_train:
            label = int(e.pop(-1))
        review = " ".join(e[1:])
        review = re.sub(r"^\"*", "", review)
        review = re.sub(r"\"*$", "", review)
        review = preprocess(review)
        review_ws = rdrsegmenter.tokenize(review)
        review_ws = ' '.join([' '.join(x) for x in review_ws])
        if is_train:
            lst.append([idx, review, review_ws, label])
        else:
            lst.append([idx, review, review_ws])
    return lst

In [ ]:
def load_csv_data(filepath, textcol="text"):
    df = pd.read_csv(filepath)
    samples = [ str(text) for text in df[textcol] ]
    labels  = [ intent for intent in df["label"] ]

    return samples, labels

In [ ]:
train_data = load_data(TRAIN_FILE)
test_data = load_data(TEST_FILE, is_train=False)

print("# Loaded training samples: {}".format(len(train_data)))
print("# Loaded test samples: {}".format(len(test_data)))

# Loaded training samples: 16087
# Loaded test samples: 10981


In [ ]:
cols = ["id", "text", "text_ws", "label"]
colsTest = ["id", "text", "text_ws"]
df_train = pd.DataFrame(data=train_data, columns=cols)
df_train.to_csv(TRAIN_CSV, index=False, quoting=csv.QUOTE_NONNUMERIC)
df_test = pd.DataFrame(data=test_data, columns=colsTest)
df_test.to_csv(TEST_CSV, index=False, quoting=csv.QUOTE_NONNUMERIC)

In [ ]:
train_labels = [tp[-1] for tp in train_data]
dev_train_data, val_data, _, _ = train_test_split(train_data,
                                                  train_labels, test_size=0.2)
df_train_data = pd.DataFrame(data=dev_train_data, columns=cols)
df_val_data = pd.DataFrame(data=val_data, columns=cols)

df_train_data.to_csv(DEV_TRAIN_CSV, index=False, quoting=csv.QUOTE_NONNUMERIC)
df_val_data.to_csv(VAL_CSV, index=False, quoting=csv.QUOTE_NONNUMERIC)

In [ ]:
df_train_data

,id,text,text_ws,label
0,train_003502,không ngon lắm khô mềm và có mùi. mình,không ngon lắm khô mềm và có mùi . mình,1
1,train_015999,chất lượng sản phẩm kém. 1 ngày dùng chết 3 lần,chất_lượng sản_phẩm kém . 1 ngày dùng chết 3 lần,1
2,train_011762,chất lượng sản phẩm rất kém quá tệ . mã không ...,chất_lượng sản_phẩm rất kém quá tệ . mã không ...,1
3,train_014280,quần áo bị rách mà vẫn giao cho khách,quần_áo bị rách mà vẫn giao cho khách,1
4,train_005364,trả tiền rồi. không cho đồng kiểm. kiểm xong...,trả tiền rồi . không cho đồng kiểm . kiểm xong...,1
...,...,...,...,...
12864,train_015122,10 năm 1 thiết kế,10 năm 1 thiết_kế,1
12865,train_004011,tốt chức năng 🤗,tốt chức_năng 🤗,0
12866,train_002737,hoàn toàn không hài lòng,hoàn_toàn không hài_lòng,1
12867,train_001619,chất lượng sản phẩm tuyệt vời mùi rất dễ chịu,chất_lượng sản_phẩm tuyệt_vời mùi rất dễ_chịu,0


In [ ]:
df_val_data

,id,text,text_ws,label
0,train_010577,mình mua được 1 tháng thì không nhận sim nữa. ...,mình mua được 1 tháng thì không nhận sim nữa ....,1
1,train_010588,chất lượng sản phẩm tuyệt vời cửa hàng phục vụ...,chất_lượng sản_phẩm tuyệt_vời cửa_hàng phục_vụ...,0
2,train_010220,rẻ tiền có khác châdt liệu tạm ổn hơi mỏng thô...,rẻ_tiền có khác châdt liệu tạm ổn hơi mỏng thô...,1
3,train_003340,đặt 2 cái cùng size cửa hàng chuyển cho mình 2...,đặt 2 cái cùng size cửa_hàng chuyển cho mình 2...,0
4,train_004197,chất lượng sản phẩm tuyệt vời rất đáng tiền. t...,chất_lượng sản_phẩm tuyệt_vời rất đáng tiền . ...,0
...,...,...,...,...
3213,train_003903,chất lượng sản phẩm rất kém rất không đáng tiề...,chất_lượng sản_phẩm rất kém rất không đáng tiề...,1
3214,train_010914,chất lượng sản phẩm rất kém .đóng gói sản phẩm...,chất_lượng sản_phẩm rất kém . đóng_gói sản_phẩ...,1
3215,train_007604,chất lượng sản phẩm rất kém đóng gói sản phẩm ...,chất_lượng sản_phẩm rất kém đóng_gói sản_phẩm ...,1
3216,train_000764,đóng gói sản phẩm rất đẹp và chắc chắn giá tốt...,đóng_gói sản_phẩm rất đẹp và chắc_chắn giá tốt...,0


# Model

In [ ]:
C_OPTIONS = np.logspace(-3, 3, 10)
C_OPTIONS

array([1.00000000e-03, 4.64158883e-03, 2.15443469e-02, 1.00000000e-01,
       4.64158883e-01, 2.15443469e+00, 1.00000000e+01, 4.64158883e+01,
       2.15443469e+02, 1.00000000e+03])

In [ ]:
def train(train_sens, y_train, tune_sgd=False):
    C_OPTIONS = np.logspace(-3, 3, 10)
    text_clf = Pipeline([('vect', CountVectorizer(ngram_range=(1,3))),
                         ('tfidf', TfidfTransformer(sublinear_tf=False)),
                         ('clf', LinearSVC(dual=False, tol=1e-3, penalty="l2", loss='squared_hinge'))])
    parameters = {
        'clf__C': C_OPTIONS,
    }

    score = 'f1'

    print("\nTuning parameters for Linear SVM\n")
    gs_clf = GridSearchCV(text_clf, parameters, cv=5, scoring=score,
                          n_jobs=1)
    gs_clf.fit(train_sens, y_train)
    for param_name in sorted(parameters.keys()):
        print("%s: %r" % (param_name, gs_clf.best_params_[param_name]))
    print("Best score (Grid search SVC): %f" % gs_clf.best_score_)

    model1 = LinearSVC(
        C=gs_clf.best_params_['clf__C'],
        loss='squared_hinge',
        penalty="l2",
        dual=False, tol=1e-3)

    if tune_sgd:

        print("\nTunning parameters for SGDClassifier\n")

        text_clf = Pipeline([('vect', CountVectorizer(ngram_range=(1,3))),
                             ('tfidf', TfidfTransformer(sublinear_tf=False)),
                             ('clf', SGDClassifier(alpha=0.0001, n_jobs=1))])

        parameters = {
            'clf__l1_ratio': [0.01, 0.1, 0.15, 0.25, 0.3],
            'clf__loss': ['hinge', 'log', 'squared_hinge'],
            'clf__penalty': ['l1', 'l2', 'elasticnet'],
        }
        gs_clf = GridSearchCV(text_clf, parameters, cv=5, scoring=score,
                              n_jobs=-1)
        gs_clf.fit(train_sens, y_train)
        for param_name in sorted(parameters.keys()):
            print("%s: %r" % (param_name, gs_clf.best_params_[param_name]))
        print("Best score (Grid search SGD): %f" % gs_clf.best_score_)

        model2 = SGDClassifier(alpha=0.0001,
                             l1_ratio=gs_clf.best_params_['clf__l1_ratio'],
                             loss=gs_clf.best_params_['clf__loss'],
                             penalty=gs_clf.best_params_['clf__penalty'])
    else:
        model2 = SGDClassifier(alpha=0.0001, l1_ratio=0.1, loss='hinge', penalty="l2")

    print("\nEnsemble learning\n")

    model3 = RandomForestClassifier(n_estimators=200, max_depth=None,
                                  random_state=100)

    ensemble_clf = VotingClassifier(
        estimators=[('linearSVM', model1), ('sgd', model2),
                    ('rdf', model3),
                    ],
        voting='hard')

    ensembleModel = Pipeline([('vect', CountVectorizer(ngram_range=(1,3))),
                         ('tfidf', TfidfTransformer(sublinear_tf=False)),
                         ('clf', ensemble_clf),])

    ensembleModel.fit(train_sens, y_train)
    return ensembleModel

In [ ]:
def predict(ensembleModel, test_sens):
    return ensembleModel.predict(test_sens)

In [ ]:
train_samples, train_labels = load_csv_data(DEV_TRAIN_CSV, textcol="text_ws")
test_df = pd.read_csv(VAL_CSV)
test_indices = [str(text) for text in test_df["id"]]
test_samples = [str(text) for text in test_df["text_ws"]]
test_labels = [intent for intent in test_df["label"]]

In [ ]:
ensembleModel = train(train_samples, train_labels, tune_sgd=True)


Tuning parameters for Linear SVM

clf__C: 2.154434690031882
Best score (Grid search SVC): 0.890524

Tunning parameters for SGDClassifier

clf__l1_ratio: 0.25
clf__loss: 'hinge'
clf__penalty: 'l2'
Best score (Grid search SGD): 0.891179

Ensemble learning



In [ ]:
preds = predict(ensembleModel, test_samples)

In [ ]:
preds[0:20]

array([1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1])

In [ ]:
test_labels[0:20]

[1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1]

In [ ]:
acc = metrics.f1_score(test_labels, preds)
print("Accuracy: {}".format(acc))
print(metrics.classification_report(test_labels, preds))

Accuracy: 0.8926916221033867
              precision    recall  f1-score   support

           0       0.95      0.88      0.92      1884
           1       0.85      0.94      0.89      1334

    accuracy                           0.91      3218
   macro avg       0.90      0.91      0.90      3218
weighted avg       0.91      0.91      0.91      3218



# Save model and load

In [ ]:
SAVE_MODEL_PATH = "/content/drive/MyDrive/Colabs/saveModel/ensembleModelNewData.pt"
torch.save(ensembleModel, SAVE_MODEL_PATH)

In [ ]:
modelSave = torch.load(SAVE_MODEL_PATH)
modelSave

Pipeline(steps=[('vect', CountVectorizer(ngram_range=(1, 3))),
                ('tfidf', TfidfTransformer()),
                ('clf',
                 VotingClassifier(estimators=[('linearSVM',
                                               LinearSVC(C=2.154434690031882,
                                                         dual=False,
                                                         tol=0.001)),
                                              ('sgd',
                                               SGDClassifier(l1_ratio=0.25)),
                                              ('rdf',
                                               RandomForestClassifier(n_estimators=200,
                                                                      random_state=100))]))])

# Classtify

In [ ]:
listToTest = df_test["text_ws"].tolist()

In [ ]:
idCheck = 30
senToCheck = listToTest[idCheck]
print("-------------------")
print("Câu bình luận là: \n"+ senToCheck)
predictSen = predict(modelSave, [senToCheck])
print("-------------------")
if predictSen:
    print("Đây là bình luận tiêu cực")
else:
    print("Đây là bình luận tích cực")

-------------------
Câu bình luận là: 
xe_hơi nhỏ cũng được
-------------------
Đây là bình luận tiêu cực
